## Mask R-CNN - Train

In [13]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print("ROOT_DIR", ROOT_DIR)

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

ROOT_DIR /home/akeaveny/catkin_ws/src/object-rpe-ak/Mask_RCNN/samples/parts_affordance


## Configurations

In [15]:
import dataset_syn_hammer as Affordance
config = Affordance.AffordanceConfig()

## Notebook Preferences

In [10]:
############################################################
#  Parse command line arguments
############################################################

args = dict()

args['train'] = 'rgbd' ### rgb or rgbd
args['dataset'] = '/data/Akeaveny/Datasets/part-affordance_combined/ndds2/'
args['weights'] = 'coco'
args['logs'] = DEFAULT_LOGS_DIR

print(args)

rgbd


In [11]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

from mrcnn.config import Config
if args['train'] == 'rgb':
    from mrcnn import model as modellib, utils
if args['train'] == 'rgbd':
    from mrcnn import modeldepth as modellib, utils

In [12]:
def train(model, args):

    """Train the model."""
    # Training dataset.
    dataset_train = Affordance.AffordanceDataset()
    dataset_train.load_Affordance(args.dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = Affordance.AffordanceDataset()
    dataset_val.load_Affordance(args.dataset, "val")
    dataset_val.prepare()

    if args.display_keras:
        model.keras_model.summary()
    config.display()

    ##################
    #  IMMGAUG
    ##################

    augmentation = iaa.Sometimes(0.5, [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Multiply((0.8, 1.2)),
        iaa.GaussianBlur(sigma=(0.0, 2.0)),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
    ])

    # elif args.dataset_type == 'syn' or args.dataset_type == 'syn1':
    #    augmentation = None

    #############################
    #  Learning Rate Scheduler
    #############################

    # Training - Stage 1 HEADS
    # HEADS
    print("\n************* trainining HEADS *************")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10, # 100
                augmentation=augmentation,
                layers='heads')

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("\n************* trainining ResNET 4+ *************")
    model.train(dataset_train, dataset_val,
              learning_rate=config.LEARNING_RATE,
              epochs=15,  # 100
              augmentation=augmentation,
              layers='4+')

    # Training - Stage 3
    # Fine tune all layers
    print("\n************* trainining ALL *************")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/10,
                epochs=20, # 240
                augmentation=augmentation,
                layers='all')

## Load Model

In [5]:
### if __name__ == '__main__':
with tf.device(DEVICE):
  
    print("Weights: ", args.weights)
    print("Dataset: ", args.dataset)
    print("Logs: ", args.logs)

    # Configurations
    config = Affordance.AffordanceConfig()

    # Create model
    model = modellib.MaskRCNN(mode="training", config=config,
                                model_dir=args.logs)

    # Select weights file to load
    if args.weights.lower() == "coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif args.weights.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    elif args.weights.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    else:
        weights_path = args.weights

    # Load weights
    print("Loading weights ", weights_path)
    if args.weights.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    else:
        model.load_weights(weights_path, by_name=True)

    # Train
    train(model, args)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


Loading weights  /home/akeaveny/catkin_ws/src/object-rpe-ak/Mask_RCNN/logs/hammer/affordance20200717T0510/mask_rcnn_affordance_0170.h5
Re-starting from epoch 170
